In [ ]:
import os
import re

# Preprocessing
- Replace IP addresses with "ip_addr"
- Replace url with "url"
- Lowercase everything
- Split by delimiter

In [ ]:
dirname = "data/malicious"

subnet_regex = r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}/\d{1,3}"
ip_addr_regex = r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}"
url_regex = r"(http.*?)['\"]"
output_directory="processed"
for root, subdir, files in os.walk(dirname):
    for file in files:
        content = None
        with open(f'{root}/{file}', encoding="mbcs") as f:
            content = f.read()
            content = re.sub(ip_addr_regex, "ip_addr", content)
            content = re.sub(url_regex, "url", content)
            content = content.lower()
            content = re.split(r"[;\[\s\]{}\(\)\+\.\&\"\'\,\:]+", content)
            
        with open(f'data/processed/malicious/{file}', "w", encoding="mbcs") as f:
            f.write("\n".join([c for c in content if c]))

In [ ]:
# subnet_regex = r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}/\d{1,3}"
# with open(f'data/1023.ps1') as f:
#     content = f.read()
#     matches = re.sub(subnet_regex, "subnet", content)
#     if matches:
#         print(file)
#         print(matches)

# Feature extraction
- Vector of top N most frequent words
- word2vec feature vector

In [2]:
import os
processed_directory = "data/processed/malicious"

list_of_files = []
for root, subdir, files in os.walk(processed_directory):
    for file in files:
        list_of_files.append(f'{root}/{file}')
print(list_of_files[:5])

['data/processed/malicious/1.ps1', 'data/processed/malicious/10.ps1', 'data/processed/malicious/100.ps1', 'data/processed/malicious/1000.ps1', 'data/processed/malicious/1001.ps1']


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(input="filename", decode_error="ignore")
out = vectorizer.fit_transform(list_of_files)

In [19]:
output = vectorizer.get_feature_names_out()
import collections
for row in out:
    

TypeError: unhashable type: 'csr_matrix'

In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class files:
    def __iter__(self):
        self.list_of_files = list_of_files
        self.counter = 0
        return self
    
    def __next__(self):
        if self.counter >= len(self.list_of_files):
            raise StopIteration
        
        filename = self.list_of_files[self.counter]
        with open(filename, encoding="mbcs") as f:
            content = f.read().split("\n")
        self.counter += 1
        return gensim.models.doc2vec.TaggedDocument(content, [self.counter-1])
        
    
import gensim

# model = Word2Vec(sentences=files(), vector_size=100, window=5, min_count=1, workers=4)
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=10, epochs=40)
model.build_vocab(files())
model.train(files(), total_examples=model.corpus_count, epochs=model.epochs)

2022-10-17 00:11:33,597 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d50,n5,w5,mc10,s0.001,t3>', 'datetime': '2022-10-17T00:11:33.597536', 'gensim': '4.2.0', 'python': '3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:29:51) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}
2022-10-17 00:11:33,599 : INFO : collecting all words and their counts
2022-10-17 00:11:33,601 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2022-10-17 00:11:34,986 : INFO : collected 144978 word types and 4202 unique tags from a corpus of 4202 examples and 2938062 words
2022-10-17 00:11:34,987 : INFO : Creating a fresh vocabulary
2022-10-17 00:11:35,066 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=10 retains 4527 unique words (3.12% of original 144978, drops 140451)', 'datetime': '2022-10-17T00:11:35.066800', 'gensim': '4.2.0', 'python': '3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:2

In [5]:
ranks = []
second_ranks = []
train_corpus = list(files())
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [11]:
import collections

counter = collections.Counter(ranks)
print(counter)

sum([v for k, v in counter.items() if k != 0])

Counter({0: 815, 1: 243, 2: 73, 3: 59, 4: 58, 5: 30, 6: 28, 7: 27, 10: 18, 15: 17, 12: 16, 8: 16, 25: 15, 9: 14, 14: 14, 13: 13, 17: 10, 87: 10, 29: 9, 28: 9, 86: 9, 11: 9, 36: 9, 50: 9, 80: 9, 55: 9, 120: 8, 133: 8, 35: 8, 78: 8, 297: 8, 40: 8, 21: 8, 34: 8, 777: 7, 151: 7, 31: 7, 66: 7, 16: 7, 22: 7, 83: 7, 48: 7, 72: 7, 42: 7, 19: 6, 57: 6, 158: 6, 20: 6, 47: 6, 383: 6, 77: 6, 146: 6, 183: 6, 124: 6, 227: 6, 27: 6, 102: 6, 200: 6, 70: 6, 44: 6, 554: 6, 85: 6, 82: 6, 157: 5, 224: 5, 68: 5, 52: 5, 43: 5, 37: 5, 244: 5, 145: 5, 226: 5, 233: 5, 51: 5, 367: 5, 107: 5, 538: 5, 100: 5, 140: 5, 488: 5, 45: 5, 110: 5, 60: 5, 267: 5, 314: 5, 127: 5, 271: 5, 69: 5, 24: 5, 201: 5, 38: 5, 81: 5, 116: 5, 76: 5, 75: 5, 1024: 5, 287: 5, 26: 5, 187: 5, 98: 5, 309: 5, 63: 5, 280: 5, 23: 5, 239: 5, 154: 5, 732: 5, 137: 5, 191: 5, 176: 4, 326: 4, 94: 4, 64: 4, 167: 4, 74: 4, 171: 4, 33: 4, 649: 4, 117: 4, 407: 4, 466: 4, 90: 4, 152: 4, 193: 4, 119: 4, 401: 4, 41: 4, 437: 4, 136: 4, 198: 4, 112: 4, 135:

3387